# Leverage trading with the Agave platform

fork gnosis chain 
```
ganache-cli --fork https://rpc.gnosischain.com --account="0xa0e62e06ffc813d27fd05161e2e9dbf1050c8bb2a9eb03bb21dc300eb011db27, 0xD3C21BCECCEDA1000000"
```

## imports and contracts

In [8]:
from web3 import Web3
from decouple import config
import json

RPC_Endpoint = config('FORK_RPC')
w3 = Web3(Web3.HTTPProvider(RPC_Endpoint))
account = w3.eth._get_accounts()[0]

# Constants
GWEI = 1000000000
TOKEN = 1e18

# Token Addresses
wbtc_address     = Web3.toChecksumAddress('0x8e5bbbb09ed1ebde8674cda39a0c169401db4252')
usdc_address     = Web3.toChecksumAddress('0xddafbb505ad214d7b80b1f830fccc89b60fb7a83')
wxdai_address    = Web3.toChecksumAddress('0xe91d153e0b41518a2ce8dd3d7944fa863463a97d')
link_address     = Web3.toChecksumAddress('0xe2e73a1c69ecf83f464efce6a5be353a37ca09b2')
gno_address      = Web3.toChecksumAddress('0x9c58bacc331c9aa871afd802db6379a98e80cedb')
weth_address     = Web3.toChecksumAddress('0x6a023ccd1ff6f2045c3309768ead9e68f978f6e1')
fox_address      = Web3.toChecksumAddress('0x21a42669643f45bc0e086b8fc2ed70c23d67509d')

# agToken Addresses
ag_wbtc_address  = Web3.toChecksumAddress('0x4863cfaf3392f20531aa72ce19e5783f489817d6')
ag_usdc_address  = Web3.toChecksumAddress('0x291b5957c9cbe9ca6f0b98281594b4eb495f4ec1')
ag_wxdai_address = Web3.toChecksumAddress('0xd4e420bbf00b0f409188b338c5d87df761d6c894')
ag_link_address  = Web3.toChecksumAddress('0xa286ce70fb3a6269676c8d99bd9860de212252ef')
ag_gno_address   = Web3.toChecksumAddress('0xa26783ead6c1f4744685c14079950622674ae8a8')
ag_weth_address  = Web3.toChecksumAddress('0x44932e3b1e662adde2f7bac6d5081c5adab908c6')
ag_fox_address   = Web3.toChecksumAddress('0xa916a4891d80494c6cb0b49b11fd68238aaaf617')

# Protocol Addresses
weth_gateway_address     = '0x36A644cC38Ae257136EEca5919800f364d73FeFC'
lendingpool_address      = '0x5E15d5E33d318dCEd84Bfe3F4EACe07909bE6d9c'
address_provider_address = '0x3673C22153E363B1da69732c4E0aA71872Bbb87F'
price_oracle_address     = '0x64cE22B5bA4175002AC5B6CCE3570432cA363c29'
data_provider_address    = '0x24dCbd376Db23e4771375092344f5CbEA3541FC0'

# Contract instances
lendingPool = w3.eth.contract(address=lendingpool_address, abi=json.load(open('./abis/LendingPool.json', 'r')))
wethGateway = w3.eth.contract(address=weth_gateway_address, abi=json.load(open('./abis/WethGateway.json', 'r')))
addressesProvider = w3.eth.contract(address=address_provider_address, abi=json.load(open('./abis/LendingPoolAddressesProvider.json', 'r')))
priceOracle = w3.eth.contract(address=price_oracle_address, abi=json.load(open('./abis/AgaveOracle.json', 'r')))
dataProvider = w3.eth.contract(address=data_provider_address, abi=json.load(open('./abis/AgaveDataProvider.json', 'r')))

## Deposit xDAI into Agave

In [9]:
tx_hash = wethGateway.functions.depositETH(account, 0).transact({
    'from': account, 
    'value': w3.toWei(100000, 'ether'),
    'gas_price': 10 * GWEI,
})
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("gas used: ", receipt['gasUsed'])



gas used:  224139


## Collateral and Borrow Limit

In [10]:
# [totalCollateralETH, totalDebtETH, availableBorrowsETH, currentLiquidationThreshold, ltv, healthFactor]
account_data = lendingPool.functions.getUserAccountData(account).call()
collateralInDAI = account_data[0]
availableToBorrowInDAI = account_data[2]
print("Collateral wXDAI:", w3.fromWei(collateralInDAI, 'ether'))
print("Available wXDAI: ", w3.fromWei(availableToBorrowInDAI, 'ether'))

Collateral wXDAI: 400000.005112121354839779
Available wXDAI:  320000.004089697083871823


In [11]:
def getAvailableToBorrowInAsset(asset,available):
    price = priceOracle.functions.getAssetPrice(asset).call()
    print("wBTC Price:", w3.fromWei(price, "ether"))

    return available / price

wbtc_limit = getAvailableToBorrowInAsset(wbtc_address, collateralInDAI)
print("can borrow:", wbtc_limit, "BTC")

wBTC Price: 40283.773456038483984444
can borrow: 9.929556513583309 BTC


# Notes
myContract.events.myEvent().processReceipt(receipt)
deposit_event = wethGateway.events.Deposit().processReceipt(deposit_receipt_1)
oracle = addressesProvider.functions.getPriceOracle().call()

receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
deposit_filter = lendingPool.events.Deposit.createFilter(
    fromBlock=receipt.blockNumber, 
)
print(deposit_filter.get_new_entries())
